# Path of Exile game stats data eda

dates: 2022.05.17 ~ 

This file contains an analysis processing of the game 'Path of Exile' data.

###### Origins of datas from here: 
poe_stats : https://www.kaggle.com/datasets/gagazet/path-of-exile-league-statistic

Data contains stats of 59000 players, from 4th August of 2017 and before now.

---

특정 유저들을 골라내는 위주로 분석을 진행할 노트북입니다. 이름을 이용하거나, 트위치 아이디를 이용하는 방식입니다.



# Dataset and libraries 

## data and libraries import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
# import wquantiles

# from sklearn.utils import resample
# from scipy import stats
# from scipy.stats import trim_mean
# from statsmodels import robust
# import statsmodels.api as sm
# import statsmodels.formula.api as smf
# from statsmodels.stats import power

df = pd.read_csv('poe_stats_2revisedrank.csv')

In [2]:
df.head(10)

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
1,1,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
2,1,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
3,1,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
5,2,False,False,SteelStreamerAdvantage,100,Deadeye,51acb8e6fac3cfaa7ff848151dad01b0a6f350e7525543...,4250334444,Steelmage,37,steelmage202,Hardcore Harbinger,1,0,0,1,1
6,2,False,False,Grom_WhyGodThreeCmWhy,100,Gladiator,5d8b80b2bdd30f3dba1036a6791187562758935e8d4723...,4250334444,rocklegend,22,gromick,SSF Harbinger HC,1,1,0,1,2
7,2,False,False,beobio_capitan,100,Inquisitor,65c58f9b186e1fbe9f927484f3f178cdb9ee0dc2427289...,4250334444,TishaVosemsot,40,beobio,SSF Harbinger,1,1,1,0,2
8,3,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
9,3,False,False,FutonBlewAway,100,Raider,46e56572ff6e403f45fac597240e13e9af225f06ca4e52...,4250334444,pleaGRC,40,NaN,SSF Harbinger,1,0,0,1,1


In [3]:
df.shape

(59776, 17)

# Let's unique user analyzing!

주로 이용할 컬럼은 account, id, name, twitch이다.

이전에 데이터셋을 탐색하며 보았듯, account는 유저의 계정 자체를 의미하며, id는 캐릭터 자체 고유아이디. name은 해당 캐릭터의 이름이며, twitch는 유저가 선택적으로 입력하는 자신의 twitch아이디이다. twitch 컬럼은 유일하게 Nan 값이 있었던 컬럼이기도 하다.

In [4]:
df[['account', 'name', 'id', 'twitch']].describe()

,account,name,id,twitch
count,59776,59776,59776,16300
unique,37123,58286,59776,9613
top,chuz101,_Flic_,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,thespunkylobster
freq,30,14,1,18


In [5]:
# 전체 고유 값 개수

df.nunique()

rank             14981
dead                 2
online               2
name             58286
level               48
class               26
id               59776
experience       54124
account          37123
challenges          37
twitch            9613
ladder               4
special_class        2
stat_Str             2
stat_Int             2
stat_Dex             2
stats_num            3
dtype: int64

In [6]:
# 결측 확인

df.isnull().sum()

rank                 0
dead                 0
online               0
name                 0
level                0
class                0
id                   0
experience           0
account              0
challenges           0
twitch           43476
ladder               0
special_class        0
stat_Str             0
stat_Int             0
stat_Dex             0
stats_num            0
dtype: int64

In [7]:
# 고유 값 개수 정리

print('Account: ', df['account'].nunique())
print('Name: ', df['name'].nunique())
print('Id: ', df['id'].nunique())
print('Twitch: ', df['twitch'].nunique())

Account:  37123
Name:  58286
Id:  59776
Twitch:  9613


# About 'name'
---

id는 고유하지만 name은 겹치는 것들이 분명 있었다. 캐릭터 name을 정하는데에 있어서 제한이 있는 것으로는 보이지 않는다.

간단히 살펴본다.

In [8]:
# 고유값 확인

df['name'].nunique()

58286

In [9]:
df['name'].sample(5)

42488            KukreAventurera
16943    SsfHC_BialaLadacznica_I
42900              dfgdfgdfgfdfg
21437                   Wisodoff
49996                  jaaavakul
Name: name, dtype: object

이름에서 겹치는 것들은 무엇이 있지?

아이디를 지을 수 있는 규칙은?

겹치면서 트위치 아이디가 같은 사람들도 있나 ? 그러면 같은 사람으로 유추할 수 있지 않을까?

비슷한 아이디가 또 있지 않을까? 비슷한 아이디는 어떻게?

한 사람이 다계정을 가지고 있지는 않을까?


##### 가장 확실한 실마리는  'account'이다.

# Account tell us...

In [10]:
df['account'].value_counts().sort_values(ascending=False)

chuz101       30
HQuality      29
TundraDrum    25
Hyperized     25
Ardwage       23
              ..
Qoom           1
Migiri         1
Shakaja        1
Clocksimus     1
woworpoe       1
Name: account, Length: 37123, dtype: int64

account는 총 37123개로, 랭크에 올라있는 약 6만여개의 캐릭터들 중 한 유저가 여러 캐릭터를 랭킹에 올린 것을 정황상 알 수 있는 것이다.

## same account, different id

한 유저가 만든 여러 id들에 대해 살펴보자.

In [11]:
df[['account','name', 'id', 'twitch']].groupby(by = 'account').count()

,name,id,twitch
account,,,
..A_A,6,6,0
0010,2,2,0
001010011100101110,1,1,1
007JBond,1,1,1
00fant,7,7,0
...,...,...,...
鹿島アントラーズFC_強化部_鈴木満,1,1,0
黒い太陽,1,1,1
김태연,3,3,3


In [12]:
df[['account', 'name', 'id']].groupby(['account'])['id'].count().sort_values(ascending=False).head(30)

account
chuz101             30
HQuality            29
TundraDrum          25
Hyperized           25
Ardwage             23
He_cTou_3a_MHou     20
l3ifson             20
CosmicWonder        19
Glenoidalis         19
shaar               19
flexxorzz           19
Fuzzballs           18
Elimoku             18
Develina            18
Maha_Bone           18
swayolo             18
inshole             18
flanelostopy        18
TheSpunkyLobster    18
yekarnybabaj        17
Mousseman           17
drgoatphd           17
Hellrazah           16
Seyser_Koze         16
raftner             16
pronemad            16
beuyapode           16
Ezhiel              16
Tunechis            15
turboteddy          15
Name: id, dtype: int64

한 유저가 최대 30개의 캐릭터를 랭킹에 올려놓은 것으로 보인다. chuz101이라는 유저를 보자.

In [13]:
# account = 'chuz101' are 30.

df[df['account'] == 'chuz101'].head(10)

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
10045,2516,True,False,Chuz_Sunder,84,Slayer,9e49cb7823b454812b066f3c1874efdbac477c8d18d17b...,1232028776,chuz101,25,NaN,SSF Harbinger HC,1,1,0,1,2
10242,2565,True,False,Chuz_WChief,84,Berserker,93c1f129fbbdf2fda65d42fbd13c75fa39b2f6445c836c...,1216709757,chuz101,25,NaN,SSF Harbinger HC,1,1,0,0,1
11167,2796,True,False,Chuz_VisLinkedByChance,83,Necromancer,fa59e96e7142c23f8c531b085c73107f38f879e8322578...,1146254611,chuz101,25,NaN,SSF Harbinger HC,1,0,1,0,1
12471,3123,True,False,Chuz_OroChief,82,Chieftain,fc8eddd1529977eefa4af1bf8f2f70c7e6037f8b1cc4cc...,1050328461,chuz101,25,NaN,SSF Harbinger HC,1,1,0,0,1
13523,3387,True,False,Chuz_Glad,81,Gladiator,1c96b4fcdef28c42219bd0f6663ff7efd89b779b75ba2b...,979602294,chuz101,25,NaN,SSF Harbinger HC,1,1,0,1,2
13853,3470,True,False,Chuz_Farm,81,Slayer,567d7754d24d403983d29d75c08f67960d959178215eb8...,958655183,chuz101,25,NaN,SSF Harbinger HC,1,1,0,1,2
14028,3514,False,False,Chuz_EA,81,Elementalist,06e11c02e8d8c118fdd91aa0135540d5655806da3fa370...,948202435,chuz101,25,NaN,SSF Harbinger HC,1,0,1,0,1
14110,3534,True,False,Chuz_FBTotems,81,Hierophant,b19dde189743f38138e5aa0dff1c4ff315af6c82e4a3a3...,943703466,chuz101,25,NaN,SSF Harbinger HC,1,1,1,0,2
15508,3885,True,False,Chuz_SRS,80,Necromancer,3d6234041d9d7c843932870441a89439918aea7521ee75...,865544505,chuz101,25,NaN,SSF Harbinger HC,1,0,1,0,1
19139,4795,True,False,Chuz_Zerker,77,Berserker,2c9b1740c2ebe9a24bc894c0ac28081535ca3b1038ee7b...,686077848,chuz101,25,NaN,SSF Harbinger HC,1,1,0,0,1


In [14]:
df[df['account'] == 'chuz101'].nunique()

rank             30
dead              2
online            1
name             26
level            19
class            11
id               30
experience       30
account           1
challenges        1
twitch            0
ladder            2
special_class     1
stat_Str          2
stat_Int          2
stat_Dex          2
stats_num         3
dtype: int64

In [15]:
# 어떤 이름들을 사용하고 있나?

df[df['account'] == 'chuz101'][['name', 'twitch']]

,name,twitch
10045,Chuz_Sunder,NaN
10242,Chuz_WChief,NaN
11167,Chuz_VisLinkedByChance,NaN
12471,Chuz_OroChief,NaN
13523,Chuz_Glad,NaN
13853,Chuz_Farm,NaN
14028,Chuz_EA,NaN
14110,Chuz_FBTotems,NaN
15508,Chuz_SRS,NaN
19139,Chuz_Zerker,NaN


### Commentary

이 유저는 ladder가 둘이다. **두 가지 모드에 집중하여 플레이**를 한 것으로 예상된다. 

class는 총 11로, 19개+7개 총 26가지의 직업중 **11가지 클래스**를 플레이 한 것으로 나타난다.

아이디를 보니 자신의 유니크한 아이디 'chuz'를 앞에 붙여서 플레이 한 것으로 보인다. 

이런식으로 몇 유저를 들여다 볼 수 있을 것으로 생각된다.

## same name, different id

한 가지 이름으로 유저는 어떻게 알아볼 수 있을 까

In [16]:
df[['account','name', 'id', 'twitch']].groupby(by = 'name').count()

,account,id,twitch
name,,,
AAAAAAAAAAAAAMUTALA,1,1,0
AAAAAAAAAAAAyourself,1,1,1
AAAAAAAAAAHHHHH,1,1,0
AAAAAAAAASUNDERAAAAAAAA,1,1,1
AAAAAAAAAlIENNNNNNNNN,1,1,0
...,...,...,...
๒๏๏๒,1,1,1
๓๏๓,2,2,2
๖Neferpitou๙,1,1,0


In [17]:
df[['account', 'name', 'id', 'twitch']].groupby(['name'])['id'].count().sort_values(ascending=False).head(30)

name
_Flic_             14
TundraDrum         11
LordTemplar        10
kotlecio            9
SirLalli            9
dopenesh            8
Hellrazah           8
Znoz                8
Cya                 8
Ekainen             7
Flashow             7
HarbinGer_Meng      7
Lodystek            7
HarbinGer_Mango     7
xWolhaiksong        7
This                6
Ryacchi             6
nextinthelane       6
Xiberia             6
Kokkuwi             6
uranari             6
Khob                6
Vorkath             6
VACK_HHC            6
Pingeee             6
Elki                6
Cursecoin           6
gMax                6
OvarianPillager     6
Mimic_SSF           6
Name: id, dtype: int64

같은 이름을 최대 14개의 캐릭터가 사용중이다. 이것은(_ Flic _) 한 account일까?

In [18]:
# name = '_Flic_' are 14.

df[df['name'] == '_Flic_'].head(10)

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
27206,6815,True,False,_Flic_,72,Ascendant,67bcd417afe0b89cfc308b26bf590b511180130f680583...,455996734,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
28071,7032,True,False,_Flic_,72,Ascendant,7ec0dc750d81ebacf2ca2cce9879da1d801f0631444d25...,440263744,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
28913,7243,True,False,_Flic_,71,Ascendant,9bd5774ed0f816b13e97673498d13aa82cbda6ebcea9c2...,424709581,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
36978,9263,True,False,_Flic_,68,Ascendant,72e45259ccd8f39f0582fe90c4f518bcd612b38d591bd8...,314784438,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
40020,10024,True,False,_Flic_,66,Ascendant,4f1ccb10064cfb440ac865c3c2aaa658ecbcd581374b3e...,273456954,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
41317,10349,True,False,_Flic_,65,Ascendant,11b3c4652b8198d542f3da11b08d7775b3460937a188b6...,253743465,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
42076,10539,True,False,_Flic_,65,Ascendant,e946cd6ebc3e957f39860a7d7f39efc0b632512d5b4fe5...,240160428,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
42405,10622,True,False,_Flic_,65,Ascendant,0633f8211bb7fe74aa2fba4316ee68f76db8f013af796c...,233709956,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
44372,11116,True,False,_Flic_,63,Ascendant,0003c4ccac35b1f8c69a8b5b18701bf1a7310800ac1f56...,196299619,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3
46567,11665,True,False,_Flic_,61,Ascendant,3eda4821da06924fe80ef166ff5f00fd639d4d0c611cf4...,166472598,He_cTou_3a_MHou,20,NaN,SSF Harbinger HC,1,1,1,1,3


In [19]:
df[df['name'] == '_Flic_'].nunique()

rank             14
dead              1
online            1
name              1
level            10
class             1
id               14
experience       14
account           1
challenges        1
twitch            0
ladder            1
special_class     1
stat_Str          1
stat_Int          1
stat_Dex          1
stats_num         1
dtype: int64

### Commentary

이 Flic이라는 아이디를 쓰는 유저는 account 고유 값이 1이므로 한 사람이다. twitch 아이디는 적혀있지 않지만 account가 한개인 것을 알 수 있다. 

우연으로 겹친 아이디도 분명 있을 수도 있을것이라 생각하지만 그렇지 않았다.

또한 이 사람은 아까 account == 'chuz101'인 사람과 달리, **한 모드에서 한 가지 클래스만** 집중적으로 한 것으로 보인다. 



## same twitch, different id

이번엔 같은 twitch아이디를 쓰는데 여러 캐릭터를 운영중인 유저를 찾아볼 예정이다.

어쩌면 다계정을 운영중인지에대한 여부도 알 수 있을 것이라 생각된다. (두근두근)

In [20]:
# 한 twitch 아이디로 여러 id를 운영중인 사람들

df['twitch'].value_counts().sort_values(ascending=False)

thespunkylobster    18
iispasov            16
seyser_koze         16
yoshixt             15
egheo17             15
                    ..
jigalopoe            1
taggarn              1
ragnion              1
kimitsugo            1
lecarn               1
Name: twitch, Length: 9613, dtype: int64

한 twitch 아이디 안에 있는 여러 이름과 아이디들을 보자.

가장 많은 수를 기록하고 있는 thespunkylobster 를 보도록 해보자.

In [21]:
# twitch = thespunkylobster

df[df['twitch'] == 'thespunkylobster']

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
487,122,True,False,ThroughThePrismOfSkooma,94,Raider,0be0d070caf5d4254fb2bd731c2c2b5fead2e1db19cbcf...,2806353782,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,0,1,1
495,124,True,False,Khajiit_Is_Innocent,94,Gladiator,07ba54024352737d8693d61794a1e21f7ff60d4a7c556f...,2798624220,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,1,2
1026,257,True,False,LobstersOnSkooma,93,Slayer,05c0abe82318cd4fdc20142d73b4ab4af36f24b32c94bd...,2465132591,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,1,2
2887,722,True,False,WindsOfSkooma,90,Raider,ce0931dea5c235a231be0487cd51aed0f82bf8737b1ce2...,1955389953,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,0,1,1
4965,1243,False,False,ForTheLoveOfSkomma,88,Slayer,263ea5b2089de5383dd9e0eff5d308746bc1a6c362044e...,1689357993,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,1,2
5612,1405,True,False,SpacedOutOnSkomma,87,Pathfinder,8cb6391296e8b7916cc1174195bf261d9ac4103f7abab1...,1619835792,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,0,1,1
8675,2173,False,False,SkoomaRage,85,Necromancer,ddf286fe358060d8dc3eac59e61541051b774fee6ff127...,1335048935,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,1,0,1
8744,2190,True,False,SkoomaCausesBadDecision,85,Deadeye,dfa99a355be72cc79fe54b0312b623df367ed4a05f2cd5...,1328292006,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,0,0,1,1
9659,2419,False,False,DarthSkooma,84,Berserker,db7f3565e482bdd84e0589f7c6685ad7e78273f234a9d3...,1260634487,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,0,1
10299,2579,True,False,SkoomaMeatballs,84,Slayer,d9fdf5925af585ef933e992b17a31585ca483bbc3b5d83...,1213016381,TheSpunkyLobster,30,thespunkylobster,SSF Harbinger HC,1,1,0,1,2


In [22]:
df[df['twitch'] == 'thespunkylobster'].nunique()

rank             18
dead              2
online            1
name             16
level            13
class             9
id               18
experience       18
account           1
challenges        1
twitch            1
ladder            1
special_class     1
stat_Str          2
stat_Int          2
stat_Dex          2
stats_num         2
dtype: int64

### Commentary

우선 궁금했던 account의 수는 1개이다. 다계정을 운영하지는 않는 것으로 보인다. 

이 유저의 **name 을 보면 작은 규칙**이 있다. Skooma, Skomma, Lobster(s), Khajiit 등의 표현을 넣고 있으며, 그런 표현은 한 번만 등장하지 않는다. 

이러한 점들을 보아서는 이 트위치 유저가 여러개의 캐릭터를 관리하며 랭킹에 이름을 올렸을 가능성이 있다. 

(이 트위치 아이디가 적힌 것이 해당 유저의 팬들이 따라서 쓰거나 사칭하며 랭킹에 이름을 올린게 아니라면 말이다.)

특이한 점은 이 유저는 SSF Harbinger HC 즉 **혼자 하는 Hardcore 모드**에만 등장한다는 점이다. 또한 HC모드는 dead 여부가 두 가지로 구분되는데 살아남은 상태인 것도 있는 것으로 보인다.

그런 솔로 플레이를 twitch에 방송을 하고 있었을 가능성이 있다.

또한 class는 총 9개로 18개의 캐릭터 중 겹치기도 하면서 여러 클래스를 플레이 했을 것이고, special_class컬럼이 고유가1개인데 9개 클래스인 것으로 보아 모든 캐릭터는 전직을 한 상태인 것으로 보인다.

# 한 계정에서 동시 접속이 가능한가?

In [23]:
# 한 계정안의 online 여부 개수

coinonline = df[['account', 'name', 'id', 'online']]
coinonline

,account,name,id,online
0,TheTzn,Tzn_NecroIsFineNow,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,False
1,RaizQT,RaizNeverFirstQT,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,False
2,GucciPradas,GucciStreamerAdvantage,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,False
3,Chiroxun,ChiroxPrime,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,False
4,cooltail,Cool_NecroIsFineNow,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,False
...,...,...,...,...
59771,Romanchik002,ПроклятьеРекласта,d33b4f6e08c10e365765f9a36a8f36d561fd1d86f10ee1...,False
59772,Ishibashisan,IshibashiSummoner,5764cfa387e0a87a4bebc1a3c5017e92de8bbb06445c47...,False
59773,ruby01,BLively,9ac75ab75a47cee8a9dfb0a31912df8909720a8b20f9f1...,False
59774,Lecarn,vawddvaw,cf02dfc0c90b2df9c7ac76bbedd91e93c2a8a2ca629c15...,False


In [24]:
# 상태가 두 가지 이상인 캐릭터들

coinonline = df.groupby(['account'], observed = False)['online'].sum() > 1
coinonline.sample(10)

account
OMGnewlegue      False
Rapaponu         False
Isra             False
Hermaaaaan       False
Groga            False
Sturm616         False
GradLo           False
Frozenqt         False
leChucki         False
eagleclaw1939    False
Name: online, dtype: bool

위의 coinonline은 account 별로 online들의 값들 즉 True값 합이 1초과인 경우를 가져온다.

한 account 내 online의 True가 한개인 경우는 한 account 내에서 여러 캐릭터들 중 한 캐릭터를 정상적으로 접속한 것으로 볼 수 있다는 점을 생각하여

합이 1 초과이면 한 account에서 여러 캐릭터를 접속한 상태라고 볼수 있다.

그렇다면 누가 여러 캐릭터를 동시 접속한 상태인 것일까?

In [25]:
coinonline[coinonline == True]

account
Syquiq       True
konicar45    True
nibarel      True
Name: online, dtype: bool

In [26]:
coinuser = coinonline[coinonline == True].index
coinuser

Index(['Syquiq', 'konicar45', 'nibarel'], dtype='object', name='account')

In [27]:
# 위의 캐릭터들을 데이터 셋에서 가져온다.

for i in coinuser:
    arg = df['account'] == i
    Frame = df[arg][['account', 'name', 'id', 'online']]
    print(Frame)

      account                 name  \
3640   Syquiq       Sisyphus_Spins   
19330  Syquiq  Ashura_The_Survivor   
26370  Syquiq  Parallax_The_Purple   

                                                      id  online  
3640   5206a212ad4032da92b139ae634c9c288a4ad832f4ee74...    True  
19330  bd4625d8ed1c6a71accd3906f4e77f9d5eb86f12495a5d...   False  
26370  78a3bc9934d3684168548a8f3c3b991d45f3a1e78da22d...    True  
         account                 name  \
37270  konicar45         CuraBezDecka   
42938  konicar45         MlohavaCuna_   
47761  konicar45    KuracPalacP__Sisa   
59428  konicar45  RaspaljenaCrnogorka   

                                                      id  online  
37270  f9d826d704bc413f0df33241fc5880892ea2696e9602d8...    True  
42938  50acd83c75768b3d5436a342c49dcd9a1a333846838bb2...   False  
47761  4dfb907f0fad5b2ee6583d4c81bc21f73466ec8719d5da...   False  
59428  5ddb92de80d773140a32ef15a2db9735b129a0753f3a6c...    True  
       account                     nam

한 유저가 동시에 접속이 가능한가 본 것인데, 실제로 한 유저 안에서 online이 두개인 캐릭터들이 존재했다. 

존재했다는 것만으로 이상함을 느끼는데, 실제로 문제가 되는 게임규정이라면 이 유저들을 상태로 검색해 볼 수 있을 것 같다.

# One person, many account?
---

한 계정에 최대 30개의 캐릭터도 운영중인 것으로 보면 여러 캐릭터를 키우는데 있어서 제한이 있거나 그로 인한 문제가 생길 수 있는 게임은 아닌 것으로 보인다.

보통 RPG의 경우 게임 내에서 특정 행위로 부정하게 이익을 얻는 행위를 막기위해 다계정을 막는 경우가 많다. (게임 내 경제를 위해 그런 경우가 많다.)

또한 랭킹이 기록되는 경우 실력은 그렇지 않지만 랭킹에 이름을 올리기 위해 '대리'랭크 게임을 사주하거나 사주받는 경우도 더러 있기 때문에 막는 경우가 많다.

POE에서 문제가 되는 것일지는 모르겠지만, 비슷한 데이터가 주어졌을떄 이를 분류해낼 수 있는 방법을 생각해보았다.

정확히 알 수는 없지만 정황상이며, 한 가지 조건만으로는 어뷰징 유저라고 칭할 수 없고, 여러 정황이 함께 겹치면 그런 것으로 볼 수도 있다는 것이다.

name이나 account, twitch는 사용자가 직접 정하는 부분이기 때문에 특정 단어나 뉘앙스에서 해당 유저의 아이덴티티가 드러나게 되어있다. 

1. 한 account내에서 쓰이던 비슷한 name이 아닌 **다른 name**
2. 한 account내에서 집중적으로 쓰이던 클래스가 아닌 **다른 소수의 class**
3. 한 account내에서 분포된 랭크와 달리 **튀는 rank**
4. 한 account내에서 **다른 twitch**
5. 같은 name 혹은 유사한 name이나 **다른 account**
6. 같은 name 혹은 유사한 name이나 **다른 twitch**
7. 한 twitch 내에서 **다른 account**


등등이다.

분석을 진행하기에 앞서서 분석에 쓰이기 어려운 일부 column들은 과감히 내려놓으려 한다.

(online, dead)


In [28]:
df1 = df.drop(columns = ['online', 'dead'])

In [29]:
df1.head()

,rank,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
1,1,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
2,1,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
3,1,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
4,2,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1


## twitch로 걸러내기

한 가지 twitch계정에서 여러 account를 운영중인 사람들이 있나?

In [30]:
df1.groupby(['twitch'])['account'].count()

twitch
001100010010       1
00mis              1
0815_noobhunter    1
0cxjwbu213c13      1
0lede4             1
                  ..
zyc0billyfr34k     1
zykenus            2
zyldragoon         1
zyrhon             1
zyrobyte           1
Name: account, Length: 9613, dtype: int64

In [44]:
manyaccout = df1.groupby(['twitch'])['account'].nunique() > 1
manyaccout.sample(10)

twitch
dragosavp        False
ingsmasha        False
piloboxy         False
s4bber           False
dawookie         False
poptartpwnage    False
kanelife         False
boinik           False
ceonaji          False
mjolkis          False
Name: account, dtype: bool

In [45]:
manyaccout[manyaccout == True]

twitch
aza_memes       True
easykun         True
igibek          True
jimbobkevint    True
simonthadon     True
Name: account, dtype: bool

### result

한 트위치아이디를 쓰면서 많은 account를 쓰는 사람이 5명이 걸러졌다. 

이들의 account를 보기로 하였다.

In [46]:
sametw_diffacc = manyaccout[manyaccout == True].index
sametw_diffacc

Index(['aza_memes', 'easykun', 'igibek', 'jimbobkevint', 'simonthadon'], dtype='object', name='twitch')

In [51]:
# 위의 캐릭터들을 데이터 셋에서 가져온다.

for i in sametw_diffacc:
    arg = df['twitch'] == i
    Frame = df[arg][['twitch', 'account', 'name']].sort_values('account')
    print(Frame)

          twitch    account                     name
13796  aza_memes    Aza1234  AzaSkullBeltConnoisseur
53204  aza_memes  Aza_Memes            SSFbtwXDDDDDD
        twitch         account                    name
56764  easykun  KayabaAkihikoo           NCSSF_Poddy_B
57474  easykun  KayabaAkihikoo           NCSSF_Poddy_A
38732  easykun      jahnylus14  NCSSF_EasyKunCaustic_A
       twitch account             name
7498   igibek  HurmaQ           NTKPNP
29074  igibek  HurmaQ        SosuBibas
32760  igibek  HurmaQ        СосковНет
38431  igibek  HurmaQ  CurcedPersimmon
45107  igibek  HurmaQ           NLHTTK
55884  igibek  HurmaQ           IHTMZS
367    igibek  Qwapet      Pickle_Gars
2585   igibek  Qwapet        Гуакамоля
11052  igibek  Qwapet         Qwapette
25208  igibek  Qwapet        Овсяночка
             twitch       account                     name
2820   jimbobkevint  Jimbobkevint             IFoundASword
14509  jimbobkevint  Jimbobkevint           ZoomZoomZoom__
18223  jimbobke

### Commentary

위의 twitch 아이디들은 다 계정을 사용하고 있는 twtich 유저들의 account와 캐릭터의 name들이다.

aza_memes, easykun 는 account와 name에서 각각 같은 유저일 것이라 추정되는 이름들이다.

igibek 유저는 해석하기는 어렵지만 name의 규칙성이 보이지 않았고,

jimbobkevint 유저는 RealLifePleb 라는 account 작명이 도드라지는데 따로 다른 컬럼들까지 분석이 필요해 보인다.

simonthadon 유저는 Biscos 라는 account 작명이 다른 아이디와 조금 다른 부분을 보여주는데 이또한 다른 컬럼들까지 분석이 필요해 보인다.

## account로 걸러보기

### name string splitting



In [33]:
df1[['name1', 'name2', 'name3', 'name4']] = pd.DataFrame(df1.name.str.split('_', 3).tolist())

In [35]:
df1[['name1', 'name2', 'name3', 'name4']].head(10)

,name1,name2,name3,name4
0,Tzn,NecroIsFineNow,None,None
1,RaizNeverFirstQT,None,None,None
2,GucciStreamerAdvantage,None,None,None
3,ChiroxPrime,None,None,None
4,Cool,NecroIsFineNow,None,None
5,SteelStreamerAdvantage,None,None,None
6,Grom,WhyGodThreeCmWhy,None,None
7,beobio,capitan,None,None
8,Rami,The,Negromaniac,None
9,FutonBlewAway,None,None,None


In [36]:
print(df1['name'].nunique())
print(df1['name1'].nunique())

58286
52461


In [39]:
df1['name4'].notnull().sum()

1600

각 name과 account를 자연어처리를 통해 분석을 하고 싶었지만... 꿈은 컸고 거기까지는 어려울 것 같으니 같거나 다른 것만 찾아보기로 했다.


## same name 

같은 이름인데 다른 account 를 쓰는 id가 있는지 찾아보기로 했다.

In [43]:
df1.groupby(by = 'name')['account'].nunique()

name
AAAAAAAAAAAAAMUTALA        1
AAAAAAAAAAAAyourself       1
AAAAAAAAAAHHHHH            1
AAAAAAAAASUNDERAAAAAAAA    1
AAAAAAAAAlIENNNNNNNNN      1
                          ..
๒๏๏๒                       1
๓๏๓                        1
๖Neferpitou๙               1
๛คูJชาe๛ฮี                 1
๛๑bOrbEnz๑๛                1
Name: account, Length: 58286, dtype: int64

In [ ]:
# samename_diffacc = df1.groupby(by = 'name')['account'].nunique() > 1

# manyaccout = df1.groupby(['name'])['account'].nunique() > 1
# manyaccout.sample(10)